# 8-Spots paper analysis

> This is the main notebook to be executed to reproduce the analysis of the 8-spot paper.

# How to use this notebook?

First and foremost, this notebook, and all the other notebooks linked below, can be used as a read-only resource to inspect every single step of the analysis. Furthermore, many steps of the analysis (alternative burst searches or fitting procedures) are not presented in the main paper for obvious space constraints and can be found here, where the complete analysis is provided.

In the second place, this resource can be used to re-run the analysis on your machine. You can modify parameters of burst search, or use a different fitting approach and see what's the effect of these changes on the complete analysis.

> If you are new to the notebook interface you can get a quick briefing by clicking on menu *Help* -> *User interface tour*.


# Execution Overview

This "master" notebook sequentially executes all the notebooks that perform the different steps of the analysis presented in the 8-spot paper.

We execute both standard notebook or "template notebooks". A template notebook is executed several times, each time with a different input file or burst search parameters. Template notebooks are used for the μs-ALEX analysis.

To execute other notebooks we use two helper functions that we import here:

In [1]:
import os
import time
import pandas as pd
from IPython.display import display, FileLink
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

In [2]:
def run_notebook(notebook_name):
    """Runs the notebook `notebook_name` (file name with no extension).

    This function executes notebook with name `notebook_name` (no extension)
    and saves the fully executed notebook in a new file appending "-out"
    to the original file name.

    It also displays links to the original and executed notebooks.
    """   
    timestamp_cell = "**Executed:** %s\n\n**Duration:** %d seconds."
    nb_name_full  = notebook_name + '.ipynb'
    display(FileLink(nb_name_full))
    
    out_path = 'out_notebooks/'
    out_nb_name = out_path + notebook_name + '-out.ipynb'
    
    nb = nbformat.read(nb_name_full, as_version=4)
    ep = ExecutePreprocessor(timeout = 3600)

    start_time = time.time()
    try:
        out = ep.preprocess(nb, {'metadata': {'path': './'}})
    except Exception:
        msg = 'Error executing the notebook "%s".\n\n' % notebook_name
        msg += 'See notebook "%s" for the traceback.' % out_nb_name
        print(msg)
        raise
    else:
        # On successful execution, add timestamping cell
        duration = time.time() - start_time
        timestamp_cell = timestamp_cell % (time.ctime(start_time), duration)
        nb['cells'].insert(0, nbformat.v4.new_markdown_cell(timestamp_cell))
    finally:
        nbformat.write(nb, out_nb_name)
        display(FileLink(out_nb_name))
    

def run_notebook_template(notebook_name, remove_out=True,
                          data_ids=['7d', '12d', '17d', '22d', '27d'],
                          ph_sel=None):
    """Run a template ALEX notebook for all the 5 samples.

    Fit results are saved in the folder 'results'.
    For each sample, the evaluated notebook containing both plots
    and text output is saved in the 'out_notebooks' folder.
    """
    timestamp_cell_pattern = "**Executed:** %s\n\n**Duration:** %d seconds."
    ## Compute TXT data results file name (removing a previous copy)
    assert ph_sel in ['all-ph', 'Dex', 'DexDem', 'AexAem', 'AND-gate', None]
    ph_sel_suffix = '' if ph_sel is None else '-%s' % ph_sel
    data_fname = 'results/' + notebook_name + '%s.txt' % ph_sel_suffix
    if remove_out and \
       os.path.exists(data_fname):
            os.remove(data_fname)

    nb_name_full = notebook_name + '.ipynb'
    display(FileLink(nb_name_full))

    out_path = 'out_notebooks/'
    
    ep = ExecutePreprocessor(timeout = 3600)
    for data_id in data_ids:
        nb = nbformat.read(nb_name_full, as_version=4)
        
        nb['cells'].insert(1, nbformat.v4.new_code_cell('data_id = "%s"' % data_id))
        nb['cells'].insert(1, nbformat.v4.new_code_cell('ph_sel_name = "%s"' % ph_sel))

        out_nb_name = out_path + notebook_name + '-out%s-%s.ipynb' %\
                      (ph_sel_suffix, data_id)
    
        start_time = time.time()
        try:
            out = ep.preprocess(nb, {'metadata': {'path': './'}})
        except:
            msg = 'Error executing the notebook "%s".\n\n' % notebook_name
            msg += 'See notebook "%s" for the traceback.' % out_nb_name
            print(msg)
            raise
        else:
            # On successful execution, add timestamping cell
            duration = time.time() - start_time
            timestamp_cell = timestamp_cell_pattern % (time.ctime(start_time), duration)
            nb['cells'].insert(0, nbformat.v4.new_markdown_cell(timestamp_cell))
        finally:
            nbformat.write(nb, out_nb_name)
            display(FileLink(out_nb_name))
    display(pd.read_csv(data_fname, sep="\s+").set_index('sample'))

# Architecture notes

The simple practice of systematically saving intermediate results in text files is an important step in any scientific that aims to be [computationally reproducibile](http://www.ploscompbiol.org/article/info%3Adoi%2F10.1371%2Fjournal.pcbi.1003285).

Saving intermediate results in a text file allow to easily track and find intermediate results (either manually or programmatically). Moreover, it makes easy to track regressions (or errors) when a [Version Control System](http://en.wikipedia.org/wiki/Revision_control) (i.e. [git](http://git-scm.com/)) is used.

In the present analysis, each notebook saves a selection of results in a text file in the `results` folder. These files are often used as an input by subsequent notebooks.

# Batch execution

The μs-ALEX notebooks are executed in batch from a single template notebook for the 5 samples. This allows maintaining a single "analysis recipe" that is uniformly applied to all the samples. Moreover, if the template notebook is modified, the results for all the 5 samples can be computed in one step. 

> Note that not all μs-ALEX notebooks are templates. For example the notebooks for fitting the global leakage coefficient or direct excitation are simple notebooks that read the previously saved per-sample results and compute averages.

## The notebooks links
When a template notebook is executed a set of output notebooks is generated, one for each sample. After the execution, the cell will show links to the *template notebook* (unexecuted, contains no output) and to the *output notebooks* for each sample. These output notebooks are meant to be read-only and are used to inspect the full analysis results for each sample.

## The results table
After the links, the cell will also shows a table with a row for each sample and a column for each variable saved by the template notebook. To find where a specific variable is computed, you can search the variable name in the output notebook of a specific sample (use the browser "find").

## Working on template notebooks
You can open and modify a template notebook in order to perform a different analysis. To open the template notebook click on the corresponding link (see below). Beware that notebooks with "-out" in their names are *output notebooks* not meant to be executed.

After applying the modifications (and saving) you can run the template notebook directly. In this case a default data file will be loaded. The default specified at the beginning of the notebook (look for the `data_id` variable). If the execution succeeds for a single data file you can run the notebook as a template. Just switch back to the present notebook and execute the corresponding `run_notebook_template()` cell.


# Software version

Here we keep track of all the libraries versions used for the execution.

In [3]:
%load_ext version_information

In [4]:
%version_information numpy, scipy, matplotlib, tables, pandas, lmfit, seaborn, fretbursts

 - Optimized (cython) burst search loaded.
 - Optimized (cython) photon counting loaded.
--------------------------------------------------------------
 You are running FRETBursts (version 0.5.5).

 If you use this software please cite the following paper:

   FRETBursts: An Open Source Toolkit for Analysis of Freely-Diffusing Single-Molecule FRET
   Ingargiola et al. (2016). http://dx.doi.org/10.1101/039198 

--------------------------------------------------------------


Software,Version
Python,3.5.1 64bit [GCC 4.2.1 (Apple Inc. build 5577)]
IPython,4.1.2
OS,Darwin 14.5.0 x86_64 i386 64bit
numpy,1.10.4
scipy,0.17.0
matplotlib,1.5.1
tables,3.2.2
pandas,0.18.0
lmfit,0.9.3
seaborn,0.7.0


## 1. μs-ALEX: Raw Proximity Ratio Analysis

We start the analyis by processing and fiting the raw PR values for the 5 us-ALEX measurements. 
The only correction here applied to the bursts is the background correction.

The same analysis is performed for different 
burst searches, as indicated in the notebook file name suffix:

- **all-ph**: burst search on all photons
- **Dex**: burst search on photons during Donor excitation periods
- **DexDem**: burst search on photons during Donor excitation detected in the Donor channel (donor emission)
- **AND-gate**: AND-gate burst search. The bursts are the intersection (in time) of burst from a "Dex" burst search and burst from a "AexAem" burst search.

In [5]:
nb_name = 'usALEX-5samples-PR-raw'
run_notebook_template(nb_name, ph_sel='all-ph')

/Users/anto/Google Drive/notebooks/multispot_paper/usALEX-5samples-PR-raw.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-all-ph-7d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-all-ph-12d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-all-ph-17d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-all-ph-22d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-all-ph-27d.ipynb

,n_bursts_all,n_bursts_do,n_bursts_fret,E_kde_w,E_gauss_w,E_gauss_w_sig,E_gauss_w_err,S_kde,S_gauss,S_gauss_sig,S_gauss_err,E_pr_do_kde,E_pr_do_hsm,E_pr_do_gauss,nt_mean
sample,,,,,,,,,,,,,,,
7d,1371,726,599,0.9340,0.931457,0.054452,0.002225,0.5582,0.561550,0.098080,0.004007,0.0978,0.089309,0.099816,22.293113
12d,1531,411,1083,0.7612,0.749827,0.085865,0.002609,0.5848,0.570877,0.103804,0.003154,0.1012,0.119285,0.098814,22.653599
17d,3018,560,2392,0.4914,0.485029,0.101939,0.002084,0.5358,0.558352,0.110306,0.002255,0.1008,0.098344,0.103109,23.157688
22d,2552,403,2045,0.2766,0.279934,0.070817,0.001566,0.5628,0.564196,0.112860,0.002496,0.0922,0.088573,0.091123,25.373585
27d,998,204,757,0.1958,0.195112,0.058563,0.002129,0.5540,0.583387,0.112116,0.004075,0.0820,0.096362,0.088704,19.570391


In [6]:
nb_name = 'usALEX-5samples-PR-raw'
run_notebook_template(nb_name, ph_sel='DexDem')

/Users/anto/Google Drive/notebooks/multispot_paper/usALEX-5samples-PR-raw.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-DexDem-7d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-DexDem-12d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-DexDem-17d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-DexDem-22d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-DexDem-27d.ipynb

,n_bursts_all,n_bursts_do,n_bursts_fret,E_kde_w,E_gauss_w,E_gauss_w_sig,E_gauss_w_err,S_kde,S_gauss,S_gauss_sig,S_gauss_err,E_pr_do_kde,E_pr_do_hsm,E_pr_do_gauss,nt_mean
sample,,,,,,,,,,,,,,,
7d,1686,1396,151,0.8430,0.795435,0.089812,0.007309,0.6360,0.601757,0.093175,0.007582,0.0954,0.083806,0.092120,28.148237
12d,1915,722,1092,0.7140,0.706765,0.075569,0.002287,0.5592,0.587225,0.101999,0.003087,0.0916,0.086989,0.091178,27.970313
17d,4501,1045,3276,0.4634,0.447667,0.102266,0.001787,0.5702,0.574012,0.109407,0.001911,0.0950,0.095650,0.095034,29.152810
22d,3456,580,2750,0.2642,0.265565,0.067100,0.001280,0.5634,0.570735,0.109626,0.002090,0.0810,0.084635,0.088650,31.912677
27d,1397,401,943,0.1810,0.183239,0.054042,0.001760,0.6176,0.601480,0.108081,0.003520,0.0798,0.056870,0.081928,23.297139


In [7]:
nb_name = 'usALEX-5samples-PR-raw'
run_notebook_template(nb_name, ph_sel='Dex')

/Users/anto/Google Drive/notebooks/multispot_paper/usALEX-5samples-PR-raw.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-Dex-7d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-Dex-12d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-Dex-17d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-Dex-22d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-out-Dex-27d.ipynb

,n_bursts_all,n_bursts_do,n_bursts_fret,E_kde_w,E_gauss_w,E_gauss_w_sig,E_gauss_w_err,S_kde,S_gauss,S_gauss_sig,S_gauss_err,E_pr_do_kde,E_pr_do_hsm,E_pr_do_gauss,nt_mean
sample,,,,,,,,,,,,,,,
7d,1654,956,641,0.9338,0.927817,0.053928,0.002130,0.5750,0.576817,0.100866,0.003984,0.1000,0.090002,0.104080,23.042040
12d,1692,486,1157,0.7508,0.748896,0.083924,0.002467,0.5974,0.588508,0.104495,0.003072,0.0998,0.098139,0.102115,23.504845
17d,3459,766,2588,0.4906,0.480670,0.101775,0.002001,0.5740,0.583539,0.108368,0.002130,0.1054,0.120002,0.108943,23.063124
22d,2798,501,2181,0.2752,0.280392,0.067918,0.001454,0.5768,0.583358,0.110174,0.002359,0.0962,0.074235,0.104701,26.631819
27d,1146,316,792,0.1944,0.194729,0.057172,0.002032,0.5786,0.605397,0.108735,0.003864,0.0904,0.091578,0.093993,19.882093


In [8]:
nb_name = 'usALEX-5samples-PR-raw-AND-gate'
run_notebook_template(nb_name)

/Users/anto/Google Drive/notebooks/multispot_paper/usALEX-5samples-PR-raw-AND-gate.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-AND-gate-out-7d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-AND-gate-out-12d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-AND-gate-out-17d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-AND-gate-out-22d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-AND-gate-out-27d.ipynb

,n_bursts_all,n_bursts_fret,E_kde_w,E_gauss_w,E_gauss_w_sig,E_gauss_w_err,S_kde,S_gauss,S_gauss_sig,S_gauss_err,nt_mean
sample,,,,,,,,,,,
7d,667,611,0.9358,0.929582,0.053343,0.002158,0.5766,0.570903,0.098088,0.003968,23.186875
12d,1130,1104,0.7524,0.750899,0.080989,0.002437,0.5920,0.582839,0.100966,0.003039,23.409028
17d,2479,2455,0.4912,0.482385,0.099163,0.002001,0.5742,0.575934,0.102965,0.002078,23.032493
22d,2197,2110,0.2746,0.279620,0.068639,0.001494,0.5698,0.577890,0.108504,0.002362,26.565893
27d,781,760,0.1940,0.193593,0.058397,0.002118,0.5808,0.599790,0.106323,0.003857,19.687144


## 2. μs-ALEX: Leakage coefficient fit

In this section we compute the spectral leakage coefficient from the Donor-only peak positions in the Raw PR histograms.

The computation is performed by [usALEX - Corrections - Leakage fit](usALEX - Corrections - Leakage fit.ipynb)
and uses results from [usALEX-5samples-PR-raw-DexDem](usALEX-5samples-PR-raw-DexDem.ipynb) executed in the previous section.

In [9]:
run_notebook('usALEX - Corrections - Leakage fit')

/Users/anto/Google Drive/notebooks/multispot_paper/usALEX - Corrections - Leakage fit.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX - Corrections - Leakage fit-out.ipynb

## 3. μs-ALEX: Direct excitation fit

In this section we extract the direct excitation parameter by fitting the S peak position of the Acceptor-only population.

For comparison 2 burst searches are performed:

- **all-ph**: burst search on all photons
- **AexAem**: burst search on photons during Acceptor excitation detected by the Acceptor detector (acceptor emission).

The **all-ph** burst search provides less accurate results and is performed only to show the improvement of the more appropriate **AexAem** burst search.

### 3.1 Fitting of the S peak

In [10]:
nb_name = 'usALEX-5samples-PR-raw-dir_ex_aa-fit'
run_notebook_template(nb_name, ph_sel='all-ph')

/Users/anto/Google Drive/notebooks/multispot_paper/usALEX-5samples-PR-raw-dir_ex_aa-fit.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-dir_ex_aa-fit-out-all-ph-7d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-dir_ex_aa-fit-out-all-ph-12d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-dir_ex_aa-fit-out-all-ph-17d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-dir_ex_aa-fit-out-all-ph-22d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-dir_ex_aa-fit-out-all-ph-27d.ipynb

,n_bursts_aa,dir_ex_S1p,dir_ex_S_kde,dir_ex_S2p,dir_ex_S2pa,dir_ex_S2p_w1,dir_ex_S_kde_w1,dir_ex_S_kde_w4,dir_ex_S_kde_w5,dir_ex_S2p_w5,dir_ex_S2p_w5a
sample,,,,,,,,,,,
7d,393,0.055183,0.067920,0.071567,0.048074,0.064032,0.067920,0.062473,0.061346,0.055097,0.046819
12d,169,0.048452,0.072961,0.098835,0.046837,0.091981,0.072731,0.069519,0.067692,0.076268,0.042248
17d,371,0.048789,0.084363,0.106637,0.060284,0.094681,0.083893,0.072501,0.065190,0.072484,0.046313
22d,331,0.047779,0.078516,0.090018,0.058578,0.083249,0.077818,0.070435,0.064509,0.073620,0.051089
27d,61,0.035500,0.040366,0.121219,0.020701,0.075456,0.040366,0.045369,0.032631,0.035625,0.003425


In [11]:
nb_name = 'usALEX-5samples-PR-raw-dir_ex_aa-fit'
run_notebook_template(nb_name, ph_sel='AexAem')

/Users/anto/Google Drive/notebooks/multispot_paper/usALEX-5samples-PR-raw-dir_ex_aa-fit.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-dir_ex_aa-fit-out-AexAem-7d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-dir_ex_aa-fit-out-AexAem-12d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-dir_ex_aa-fit-out-AexAem-17d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-dir_ex_aa-fit-out-AexAem-22d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-raw-dir_ex_aa-fit-out-AexAem-27d.ipynb

,n_bursts_aa,dir_ex_S1p,dir_ex_S_kde,dir_ex_S2p,dir_ex_S2pa,dir_ex_S2p_w1,dir_ex_S_kde_w1,dir_ex_S_kde_w4,dir_ex_S_kde_w5,dir_ex_S2p_w5,dir_ex_S2p_w5a
sample,,,,,,,,,,,
7d,630,0.035049,0.061121,0.068715,0.060454,0.065912,0.060895,0.055075,0.053741,0.055369,0.050042
12d,269,0.037100,0.060445,0.092479,0.041670,0.073575,0.060445,0.057082,0.054852,0.057560,0.044884
17d,608,0.034677,0.083424,0.119491,0.070370,0.094961,0.082720,0.077354,0.071352,0.072895,0.174377
22d,433,0.026463,0.086484,0.111094,0.053768,0.082611,0.085541,0.071582,0.064283,0.061879,0.046918
27d,129,0.019777,0.050862,0.078734,0.040582,0.080239,0.050420,0.074807,0.043406,0.055485,0.017062


### 3.2 Computation of the direct excitation coefficient

The notebook [usALEX - Corrections - Direct excitation fit](usALEX - Corrections - Direct excitation fit.ipynb) computes the direct excitation coefficient performing an average of the values extracted from each single measurement:

In [14]:
run_notebook('usALEX - Corrections - Direct excitation fit')

/Users/anto/Google Drive/notebooks/multispot_paper/usALEX - Corrections - Direct excitation fit.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX - Corrections - Direct excitation fit-out.ipynb

## 4. μs-ALEX: $\gamma$-factor fit

In this section we compute the $\gamma$-factor for the μs-ALEX measurements.

First we reprocess the μs-ALEX measurements including background, leakage and direct excitation correction (standard PR). Then, for the fitted $E$ and $S$ peak position for the FRET population we fit the $\gamma$-factor.

### 4.1: Proximity Ratio analysis

In [18]:
nb_name = 'usALEX-5samples-PR-leakage-dir-ex-all-ph'
run_notebook_template(nb_name)

/Users/anto/Google Drive/notebooks/multispot_paper/usALEX-5samples-PR-leakage-dir-ex-all-ph.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-leakage-dir-ex-all-ph-out-7d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-leakage-dir-ex-all-ph-out-12d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-leakage-dir-ex-all-ph-out-17d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-leakage-dir-ex-all-ph-out-22d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-PR-leakage-dir-ex-all-ph-out-27d.ipynb

,n_bursts_all,n_bursts_do,n_bursts_fret,E_kde_w,E_gauss_w,E_gauss_w_sig,E_gauss_w_err,S_kde,S_gauss,S_gauss_sig,S_gauss_err,E_pr_do_kde,nt_mean
sample,,,,,,,,,,,,,
7d,1172,586,543,0.9304,0.925842,0.059539,0.002555,0.5512,0.550639,0.099496,0.004270,2.000000e-03,22.227651
12d,1307,329,948,0.7440,0.731546,0.091896,0.002985,0.5748,0.557524,0.105052,0.003412,1.600000e-02,22.002177
17d,2489,463,1964,0.4320,0.427115,0.113675,0.002565,0.5434,0.537124,0.112882,0.002547,1.280000e-02,21.192847
22d,2054,319,1673,0.1804,0.184215,0.078951,0.001930,0.5414,0.542855,0.114988,0.002811,5.717649e-15,23.016139
27d,791,160,590,0.0844,0.085181,0.070168,0.002889,0.5160,0.556065,0.115220,0.004744,-8.800000e-03,16.829689


### 4.2 $\gamma$-factor fit

The notebook [usALEX - Corrections - Gamma factor fit](usALEX - Corrections - Gamma factor fit.ipynb)
fits the $\gamma$-factor from $E$ and $S$ values computed in the previous section.

In [24]:
run_notebook('usALEX - Corrections - Gamma factor fit')

/Users/anto/Google Drive/notebooks/multispot_paper/usALEX - Corrections - Gamma factor fit.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX - Corrections - Gamma factor fit-out.ipynb

In [22]:
!say done

## 5. μs-ALEX: Corrected E analysis

Using all the correction coefficients here we computed the corrected $E$ histograms:

In [25]:
nb_name = 'usALEX-5samples-E-corrected-all-ph'
run_notebook_template(nb_name)

/Users/anto/Google Drive/notebooks/multispot_paper/usALEX-5samples-E-corrected-all-ph.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-E-corrected-all-ph-out-7d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-E-corrected-all-ph-out-12d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-E-corrected-all-ph-out-17d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-E-corrected-all-ph-out-22d.ipynb

/Users/anto/Google Drive/notebooks/multispot_paper/out_notebooks/usALEX-5samples-E-corrected-all-ph-out-27d.ipynb

,n_bursts_all,n_bursts_do,n_bursts_fret,E_kde_w,E_gauss_w,E_gauss_w_sig,E_gauss_w_err,S_kde,S_gauss,S_gauss_sig,S_gauss_err,E_pr_do_kde,nt_mean
sample,,,,,,,,,,,,,
7d,1172,587,542,0.9290,0.924307,0.060130,0.002583,0.5520,0.550744,0.099172,0.004260,2.200000e-03,22.227651
12d,1307,329,948,0.7398,0.727043,0.092062,0.002990,0.5758,0.558745,0.105387,0.003423,1.540000e-02,22.002177
17d,2489,464,1964,0.4268,0.422624,0.112952,0.002549,0.5462,0.539571,0.112486,0.002538,1.240000e-02,21.192847
22d,2054,320,1672,0.1774,0.181067,0.077181,0.001888,0.5458,0.547034,0.114990,0.002812,5.717649e-15,23.016139
27d,791,160,589,0.0826,0.083971,0.069304,0.002856,0.5208,0.561290,0.114456,0.004716,-8.800000e-03,16.829689


## 6. Direct excitation: physical parameter

Here we compute the "physical" direct direct excitation coefficient:

$$ d_{exT} = \frac{\sigma_{A532}}{\sigma_{D532}}$$

This coefficient is based only on physical dyes properties and it can therefore applied to non-ALEX measurements. For details see:

- [usALEX - Corrections - Direct excitation physical parameter](usALEX - Corrections - Direct excitation physical parameter.ipynb)

In [17]:
run_notebook('usALEX - Corrections - Direct excitation physical parameter')

C:\Data\Antonio\software\src\multispot_paper - new\usALEX - Corrections - Direct excitation physical parameter.ipynb

C:\Data\Antonio\software\src\multispot_paper - new\out_notebooks\usALEX - Corrections - Direct excitation physical parameter-out.ipynb

## 7. Multi-spot analysis

### 7.1 DCR and background

Plot background fits:

In [18]:
run_notebook('Multi-spot 5-Samples analysis - Background')

C:\Data\Antonio\software\src\multispot_paper - new\Multi-spot 5-Samples analysis - Background.ipynb

C:\Data\Antonio\software\src\multispot_paper - new\out_notebooks\Multi-spot 5-Samples analysis - Background-out.ipynb

Compute detectors DCR:

In [19]:
run_notebook('8-pixel DCR')

C:\Data\Antonio\software\src\multispot_paper - new\8-pixel DCR.ipynb

C:\Data\Antonio\software\src\multispot_paper - new\out_notebooks\8-pixel DCR-out.ipynb

### 7.2 Leakage coefficient

First step of the analysis of the multi-spot data is fitting the leakage coefficient. This is perfomed by the notebook

* [Multi-spot 5-Samples analysis - Leakage coefficient fit](Multi-spot 5-Samples analysis - Leakage coefficient fit.ipynb)

that outputs these 3 text files:

- [Multi-spot - leakage coefficient Dem.txt](results/Multi-spot - leakage coefficient Dem.txt) *(all-ch, all-samples mean)*
- [Multi-spot - leakage coefficient mean per-sample Dem.txt](results/Multi-spot - leakage coefficient mean per-sample Dem.txt)
- [Multi-spot - leakage coefficient mean per-ch Dem.txt](results/Multi-spot - leakage coefficient mean per-ch Dem.txt)

In [20]:
run_notebook('Multi-spot 5-Samples analysis - Leakage coefficient fit')

C:\Data\Antonio\software\src\multispot_paper - new\Multi-spot 5-Samples analysis - Leakage coefficient fit.ipynb

C:\Data\Antonio\software\src\multispot_paper - new\out_notebooks\Multi-spot 5-Samples analysis - Leakage coefficient fit-out.ipynb

### 7.3 PR and FRET analysis

Then we perform the complete analysis by running:

* [Multi-spot 5-Samples analysis - PR-raw](Multi-spot 5-Samples analysis - PR-raw.ipynb) 

In [21]:
run_notebook('Multi-spot 5-Samples analysis - PR-raw')

C:\Data\Antonio\software\src\multispot_paper - new\Multi-spot 5-Samples analysis - PR-raw.ipynb

Error executing the notebook "Multi-spot 5-Samples analysis - PR-raw".

See notebook "out_notebooks/Multi-spot 5-Samples analysis - PR-raw-out.ipynb" for the traceback.


C:\Data\Antonio\software\src\multispot_paper - new\out_notebooks\Multi-spot 5-Samples analysis - PR-raw-out.ipynb

CellExecutionError: An error occurred while executing the following cell:
------------------
stop
------------------

NameError: name 'stop' is not defined
